In [1]:
from keras.models import load_model
# returns a compiled model
# identical to the previous one
model = load_model('lstm_model.h5')

Using Theano backend.


In [2]:
lines = []
with open('C:\Users\pooja\Desktop\intern\IMDB-test-data.txt', 'r') as file:
    for line in file:   # reads line by line
        line = line.strip() # to remove \n
        lines.append(line)
lines[:10]

['1\tBrokeback Mountain was so awesome.',
 '1\tI loved Mission Impossible-3..',
 '0\tDa Vinci Code sucked..',
 '1\tBrokeback Mountain was so awesome.',
 '1\tNo... but I love Harry Potter...',
 '1\t"I liked the first "" Mission Impossible."',
 '0\t"Not because I hate Harry Potter, but because I am the type of person that likes it when the main character dies."',
 '0\tfriday hung out with kelsie and we went and saw The Da Vinci Code SUCKED!!!!!',
 '0\t"by the way, the Da Vinci Code sucked, just letting you know..."',
 '1\tI love The Da Vinci Code...']

In [3]:
Y = [lines[y].split("\t")[0] for y in range(len(lines))]

X = [lines[y].split("\t")[1] for y in range(len(lines))]

In [4]:
import re
def clean_str(string):

    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [5]:
new_X = [clean_str(s) for s in X]
new_X[:5]

['brokeback mountain was so awesome',
 'i loved mission impossible 3',
 'da vinci code sucked',
 'brokeback mountain was so awesome',
 'no but i love harry potter']

In [6]:
new_X = [s.split(' ') for s in new_X]         # new_X is a list of list
print new_X[1]

['i', 'loved', 'mission', 'impossible', '3']


In [7]:
from spacy.en import English
nlp = English()

In [8]:
import numpy as np
def build_input_data_nlp(sentences, labels):
    # uses spacy.en and English()
    
    nb_sentences = len(sentences)          # no. of sentences        
    nb_tokens = len(sentences[0])          # = 50 in this case
    delist = sentences[0]
    word_vec_dim = nlp(delist[0].decode('utf8'))[0].vector.shape[0]       #taken 1st word of 1st sentence
    # .vector is a 1-dimensional numpy array of 32-bit floats. The default English model installs vectors for one million vocabulary entries, using the 300-dimensional vectors
    # shape attribute for numpy arrays returns the dimensions of the array. If Y has n rows and m columns, then Y.shape is (n,m). So Y.shape[0] is n
    sentences_matrix = np.zeros((nb_sentences, nb_tokens, word_vec_dim))       # np.zero() Return a new array of given shape and type, filled with zeros
    print sentences_matrix.shape
    for k in xrange(nb_sentences):
        delist = sentences[k]
        m = len(delist)
        for i in xrange(len(delist)):      # xrange is faster compare to range
            tokens = nlp(delist[i].decode('utf8'))       # token = each word (in loop)
            for j in xrange(len(tokens)):
                sentences_matrix[k, i, :] = tokens[j].vector
    ss = sentences_matrix.shape
    y = np.array(labels)
    x = sentences_matrix
    return [x,y]

In [9]:
# counting max size of a line
max_len = 0
for itn in range(len(Y)):
    if max_len < len(new_X[itn]) :
        max_len = len(new_X[itn])
        
seq_len = 41
padding_word = '<PAD/>'
padded_sentences = []
Z = Y

for i in range(len(new_X)):
    sentence = new_X[i]
    num_padding = seq_len - len(sentence)
    new_sentence = sentence + [padding_word] * num_padding
    padded_sentences.append(new_sentence)
    
X,Y = build_input_data_nlp(padded_sentences,Y)

(1071L, 41L, 300L)


In [10]:
prediction = model.predict(X, verbose = 1)

fwrite = open('C:\Users\pooja\Desktop\intern\prediction.txt', 'w')
fwrite.write("Predicted\tActual\n")

itn = 0
count = 0.0
for itn in range(len(X)):
    if prediction[itn] > 0.5 :
        fwrite.write("1\t\t\t%s\n" %Y[itn])
        if Y[itn] == '1':
            count+=1
    else :
        fwrite.write("0\t\t\t%s\n" %Y[itn]) 
        if Y[itn] == '0':
            count+=1
print count
acc = count/len(X)
print acc

1071/1071 [==============================] - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 1s - ETA: 1s - ETA: 1s - ETA: 1s - ETA: 1s - ETA: 1s - ETA: 1s - ETA: 1s - ETA: 1s

In [11]:
print count
print acc

1038.0
0.96918767507
